# Create 1-Wave and 2-WaveFile and 3-WaveFormat json.

## Params define.

In [1]:
class Param:

    def __init__(
        self,
        file_name: str,
        num_channels: int,
        sample_rate: int,
        bits_per_sample: int,
        f0: float,
        sec: float,
    ):
        self.file_name: str = file_name  # FileName
        self.chunk_id: str = "RIFF"
        self.chunk_size: int = 0
        self.wave_format: str = "WAVE"
        self.sub_chunk_1_id: str = "fmt "
        self.sub_chunk_1_size: int = 16
        self.audio_format: int = 1  # BasicPCM
        self.num_channels: int = num_channels
        self.sample_rate: int = sample_rate  # FrameRate
        self.byte_rate: int = sample_rate * num_channels * bits_per_sample // 8
        self.block_align: int = num_channels * bits_per_sample // 8
        self.bits_per_sample: int = bits_per_sample
        self.sub_chunk_2_id: str = "data"
        self.sub_chunk_2_size: int = 0
        self.data: List[int] = None
        # Fields above this are required for json
        self.dataBytes: bytes = b""
        self.f0: float = f0  # FundamentalFrequency(Hz)
        self.sec: float = sec

    def set_data(self, data):
        self.data = data.tolist()
        self.dataBytes = data.tobytes()
        self.sub_chunk_2_size = len(self.dataBytes)
        self.chunk_size = 4 + (8 + self.sub_chunk_1_size) + (
            8 + self.sub_chunk_2_size)

In [2]:
params = [
    Param("X",
          num_channels=1,
          sample_rate=8000,
          bits_per_sample=16,
          f0=440.000,
          sec=2),
]

## 1. Create wave

In [3]:
import numpy as np

In [4]:
for p in params:
    ndarr = np.arange(0, p.sample_rate * p.sec)
    sin_wave = 32767 * np.sin(2 * np.pi * p.f0 * ndarr / p.sample_rate)
    sin_wave = sin_wave.astype(np.int16)
    p.set_data(sin_wave)

## 2. Write wav file

In [5]:
SAMPLE_WAVES_DIR = "../internal/samples/waves"

In [6]:
import wave
from os import path

In [7]:
for p in params:
    write_path = f"{path.join(SAMPLE_WAVES_DIR, p.file_name)}.wav"
    with wave.open(write_path, "wb") as f:
        f.setnchannels(p.num_channels)
        f.setsampwidth(p.block_align)
        f.setframerate(p.sample_rate)
        f.writeframes(p.dataBytes)

## 3. Write wave format json

In [8]:
SAMPLE_FORMAT_JSON_DIR = "../internal/samples/format_jsons"

In [9]:
import json

In [10]:
unnecessary_keys = ["dataBytes", "f0", "sec"]

In [11]:
for p in params:
    d = p.__dict__
    for k in unnecessary_keys:
        del d[k]
    write_path = f"{path.join(SAMPLE_FORMAT_JSON_DIR, p.file_name)}.json"
    with open(write_path, "w") as f:
        json.dump(d, f, indent=4)